In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS projeto_estatistica.gold")
print("✅ Schema Gold criado!")

✅ Schema Gold criado!


In [0]:
print("\n📈 CRIANDO AGREGAÇÕES PRINCIPAIS DE VENDAS")
print("=" * 45)

# Tabela Gold 1: Métricas de Vendas por Período
spark.sql("""
CREATE OR REPLACE TABLE projeto_estatistica.gold.vendas_por_periodo AS
SELECT 
    DATE_TRUNC('month', data_pedido) as mes,
    COUNT(*) as total_pedidos,
    ROUND(SUM(total), 2) as receita_total,
    ROUND(AVG(total), 2) as ticket_medio,
    ROUND(SUM(desconto), 2) as total_descontos,
    ROUND(SUM(subtotal), 2) as subtotal_bruto,
    COUNT(DISTINCT id_cliente) as clientes_unicos,
    -- Métricas adicionais
    SUM(CASE WHEN status_compra = 'concluido' THEN 1 ELSE 0 END) as pedidos_concluidos,
    SUM(CASE WHEN status_compra = 'cancelado' THEN 1 ELSE 0 END) as pedidos_cancelados
FROM projeto_estatistica.silver.fat_pedido_corrigida
GROUP BY DATE_TRUNC('month', data_pedido)
ORDER BY mes
""")
print("✅ vendas_por_periodo criada!")

# Mostrar resultado
print("\n📊 VENDAS POR PERÍODO:")
spark.sql("SELECT * FROM projeto_estatistica.gold.vendas_por_periodo").show(truncate=False)


📈 CRIANDO AGREGAÇÕES PRINCIPAIS DE VENDAS
✅ vendas_por_periodo criada!

📊 VENDAS POR PERÍODO:
+-------------------+-------------+-------------+------------+---------------+--------------+---------------+------------------+------------------+
|mes                |total_pedidos|receita_total|ticket_medio|total_descontos|subtotal_bruto|clientes_unicos|pedidos_concluidos|pedidos_cancelados|
+-------------------+-------------+-------------+------------+---------------+--------------+---------------+------------------+------------------+
|2025-02-01 00:00:00|278          |652942.87    |2348.72     |21.58          |699487.84     |278            |0                 |87                |
|2025-03-01 00:00:00|730          |1959397.05   |2684.11     |53.09          |2089759.96    |730            |0                 |166               |
|2025-04-01 00:00:00|657          |1704774.35   |2594.79     |47.07          |1817930.98    |657            |0                 |175               |
|2025-05-01 00:00

In [0]:
print("\n👥 CRIANDO ANÁLISE DE CLIENTES")
print("=" * 35)

# Tabela Gold 2: Métricas por Cliente
spark.sql("""
CREATE OR REPLACE TABLE projeto_estatistica.gold.metricas_cliente AS
SELECT 
    fp.id_cliente,
    dc.nome_cliente,
    dc.cidade,
    dc.estado,
    dc.regiao,
    COUNT(*) as total_pedidos,
    ROUND(SUM(fp.total), 2) as valor_total_gasto,
    ROUND(AVG(fp.total), 2) as ticket_medio,
    MIN(fp.data_pedido) as primeira_compra,
    MAX(fp.data_pedido) as ultima_compra,
    -- Segmentação por valor
    CASE 
        WHEN SUM(fp.total) > 5000 THEN 'VIP'
        WHEN SUM(fp.total) > 2000 THEN 'PREMIUM' 
        WHEN SUM(fp.total) > 500 THEN 'REGULAR'
        ELSE 'OCASIONAL'
    END as segmento_cliente
FROM projeto_estatistica.silver.fat_pedido_corrigida fp
JOIN projeto_estatistica.silver.dim_cliente dc ON fp.id_cliente = dc.id_cliente
GROUP BY fp.id_cliente, dc.nome_cliente, dc.cidade, dc.estado, dc.regiao
HAVING COUNT(*) > 0
ORDER BY valor_total_gasto DESC
""")
print("✅ metricas_cliente criada!")

# Mostrar top 10 clientes
print("\n🏆 TOP 10 CLIENTES POR VALOR:")
spark.sql("""
SELECT nome_cliente, cidade, total_pedidos, valor_total_gasto, segmento_cliente 
FROM projeto_estatistica.gold.metricas_cliente 
LIMIT 10
""").show(truncate=False)


👥 CRIANDO ANÁLISE DE CLIENTES
✅ metricas_cliente criada!

🏆 TOP 10 CLIENTES POR VALOR:
+--------------------+---------------------+-------------+-----------------+----------------+
|nome_cliente        |cidade               |total_pedidos|valor_total_gasto|segmento_cliente|
+--------------------+---------------------+-------------+-----------------+----------------+
|Maria Sophia Costela|TEFÉ                 |1            |18349.54         |VIP             |
|Augusto Da Cruz     |CARACARAÍ            |1            |18280.52         |VIP             |
|Yago Almeida        |VILHENA              |1            |18120.05         |VIP             |
|Miguel Da Luz       |SANTANA              |1            |18117.5          |VIP             |
|Júlia Moraes        |SÃO BERNARDO DO CAMPO|1            |18054.89         |VIP             |
|Murilo Almeida      |ITAJAÍ               |1            |18048.48         |VIP             |
|Sr. Samuel Teixeira |SOBRAL               |1            |17845.9 

In [0]:
print("\n📦 CRIANDO ANÁLISE DE PRODUTOS")
print("=" * 35)

# Tabela Gold 3: Performance de Produtos
spark.sql("""
CREATE OR REPLACE TABLE projeto_estatistica.gold.performance_produtos AS
SELECT 
    dp.id_produto,
    dp.nome_produto,
    dp.categoria,
    dp.subcategoria,
    dp.faixa_preco,
    COUNT(dc.id) as total_vendido,
    SUM(dc.quantidade) as unidades_vendidas,
    ROUND(SUM(dc.preco * dc.quantidade), 2) as receita_total,
    ROUND(AVG(dc.preco), 2) as preco_medio,
    -- Participação na receita
    ROUND((SUM(dc.preco * dc.quantidade) / (SELECT SUM(total) FROM projeto_estatistica.silver.fat_pedido_corrigida)) * 100, 2) as participacao_percentual
FROM projeto_estatistica.silver.dim_compra_final dc
JOIN projeto_estatistica.silver.dim_produto dp ON dc.id_produto = dp.id_produto
GROUP BY dp.id_produto, dp.nome_produto, dp.categoria, dp.subcategoria, dp.faixa_preco
ORDER BY receita_total DESC
""")
print("✅ performance_produtos criada!")

# Mostrar top 10 produtos
print("\n🏆 TOP 10 PRODUTOS POR RECEITA:")
spark.sql("""
SELECT nome_produto, categoria, unidades_vendidas, receita_total, participacao_percentual
FROM projeto_estatistica.gold.performance_produtos 
LIMIT 10
""").show(truncate=False)


📦 CRIANDO ANÁLISE DE PRODUTOS
✅ performance_produtos criada!

🏆 TOP 10 PRODUTOS POR RECEITA:
+----------------------------------------------+-----------+-----------------+-------------+-----------------------+
|nome_produto                                  |categoria  |unidades_vendidas|receita_total|participacao_percentual|
+----------------------------------------------+-----------+-----------------+-------------+-----------------------+
|Acer Notebook Gamer Nitro                     |ELETRÔNICOS|253              |1164812.0    |23.28                  |
|Projetor Epson Powerlite Wide Screen          |ELETRÔNICOS|265              |726664.45    |14.52                  |
|Projetor Smart Epson Epiqvision, Full Hd      |ELETRÔNICOS|235              |716747.65    |14.32                  |
|Samsung Galaxy A36                            |ELETRÔNICOS|243              |480921.3     |9.61                   |
|Samsung Galaxy Tab S6 Lite                    |ELETRÔNICOS|222              |399380.22

In [0]:
print("\n📊 ESTRUTURA DA ANÁLISE GEOGRÁFICA CORRIGIDA")
print("=" * 50)

print("""
🎯 AGORA TEMOS 3 NÍVEIS DE AGREGAÇÃO:

1. 🗺️ REGIÃO (vendas_por_regiao)
   • Visão mais alta - uma linha por região
   • Ideal para dashboard executivo

2. 🏛️ ESTADO (vendas_por_estado)  
   • Visão intermediária - drill down por estado
   • Permite comparar estados dentro da mesma região

3. 🏙️ CIDADE (vendas_por_localidade)
   • Visão mais detalhada - drill down por cidade
   • Para análises específicas por localidade

📈 ISSO PERMITE:
• Dashboard hierárquico (região → estado → cidade)
• Análises em diferentes níveis de granularidade
• Performance otimizada para cada tipo de consulta
""")

# Verificar a distribuição real
print("\n🔍 DISTRIBUIÇÃO REAL DE REGIÕES:")
spark.sql("""
SELECT 
    regiao,
    COUNT(DISTINCT estado) as estados,
    COUNT(DISTINCT cidade) as cidades,
    COUNT(DISTINCT id_cliente) as clientes
FROM projeto_estatistica.silver.dim_cliente
GROUP BY regiao
ORDER BY clientes DESC
""").show(truncate=False)


📊 ESTRUTURA DA ANÁLISE GEOGRÁFICA CORRIGIDA

🔍 DISTRIBUIÇÃO REAL DE REGIÕES:
+--------+-------+-------+--------+
|regiao  |estados|cidades|clientes|
+--------+-------+-------+--------+
|SUL     |3      |15     |507     |
|NORDESTE|8      |40     |500     |
|NORTE   |7      |35     |500     |
|SUDESTE |4      |19     |493     |
+--------+-------+-------+--------+



In [0]:
print("\n💳 CRIANDO ANÁLISE DE FORMAS DE PAGAMENTO")
print("=" * 45)

# Tabela Gold 5: Métricas de Pagamento
spark.sql("""
CREATE OR REPLACE TABLE projeto_estatistica.gold.metricas_pagamento AS
SELECT 
    forma_pagamento,
    COUNT(*) as total_pedidos,
    ROUND(SUM(total), 2) as valor_total,
    ROUND(AVG(total), 2) as ticket_medio,
    ROUND(AVG(desconto), 2) as desconto_medio,
    -- Distribuição percentual
    ROUND((COUNT(*) / (SELECT COUNT(*) FROM projeto_estatistica.silver.fat_pedido_corrigida)) * 100, 2) as participacao_pedidos,
    ROUND((SUM(total) / (SELECT SUM(total) FROM projeto_estatistica.silver.fat_pedido_corrigida)) * 100, 2) as participacao_valor
FROM projeto_estatistica.silver.fat_pedido_corrigida
GROUP BY forma_pagamento
ORDER BY valor_total DESC
""")
print("✅ metricas_pagamento criada!")

# Mostrar métricas de pagamento
print("\n💰 DISTRIBUIÇÃO DE PAGAMENTOS:")
spark.sql("SELECT * FROM projeto_estatistica.gold.metricas_pagamento").show(truncate=False)


💳 CRIANDO ANÁLISE DE FORMAS DE PAGAMENTO
✅ metricas_pagamento criada!

💰 DISTRIBUIÇÃO DE PAGAMENTOS:
+---------------+-------------+-----------+------------+--------------+--------------------+------------------+
|forma_pagamento|total_pedidos|valor_total|ticket_medio|desconto_medio|participacao_pedidos|participacao_valor|
+---------------+-------------+-----------+------------+--------------+--------------------+------------------+
|outros         |1003         |2422491.79 |2415.25     |0.07          |50.15               |48.41             |
|pix            |487          |1336462.1  |2744.28     |0.08          |24.35               |26.71             |
|boleto         |510          |1245214.24 |2441.6      |0.07          |25.5                |24.88             |
+---------------+-------------+-----------+------------+--------------+--------------------+------------------+



In [0]:
print("\n📋 CRIANDO RESUMO EXECUTIVO")
print("=" * 35)

# Tabela Gold 6: KPIs Principais
spark.sql("""
CREATE OR REPLACE TABLE projeto_estatistica.gold.kpis_principais AS
SELECT 
    -- KPIs de Vendas
    (SELECT COUNT(*) FROM projeto_estatistica.silver.fat_pedido_corrigida) as total_pedidos,
    ROUND((SELECT SUM(total) FROM projeto_estatistica.silver.fat_pedido_corrigida), 2) as receita_total,
    ROUND((SELECT AVG(total) FROM projeto_estatistica.silver.fat_pedido_corrigida), 2) as ticket_medio_geral,
    
    -- KPIs de Clientes
    (SELECT COUNT(DISTINCT id_cliente) FROM projeto_estatistica.silver.fat_pedido_corrigida) as total_clientes,
    ROUND((SELECT AVG(total_pedidos) FROM projeto_estatistica.gold.metricas_cliente), 2) as pedidos_por_cliente_medio,
    
    -- KPIs de Produtos
    (SELECT COUNT(DISTINCT id_produto) FROM projeto_estatistica.silver.dim_compra_final) as total_produtos,
    (SELECT SUM(unidades_vendidas) FROM projeto_estatistica.gold.performance_produtos) as total_unidades_vendidas,
    
    -- KPIs Geográficos
    (SELECT COUNT(DISTINCT cidade) FROM projeto_estatistica.silver.dim_cliente) as cidades_atendidas,
    (SELECT COUNT(DISTINCT estado) FROM projeto_estatistica.silver.dim_cliente) as estados_atendidos,
    
    -- Período de Análise
    (SELECT MIN(data_pedido) FROM projeto_estatistica.silver.fat_pedido_corrigida) as data_primeiro_pedido,
    (SELECT MAX(data_pedido) FROM projeto_estatistica.silver.fat_pedido_corrigida) as data_ultimo_pedido
""")
print("✅ kpis_principais criada!")

# Mostrar KPIs principais
print("\n🎯 KPIs PRINCIPAIS DO NEGÓCIO:")
spark.sql("SELECT * FROM projeto_estatistica.gold.kpis_principais").show(vertical=True, truncate=False)


📋 CRIANDO RESUMO EXECUTIVO
✅ kpis_principais criada!

🎯 KPIs PRINCIPAIS DO NEGÓCIO:
-RECORD 0----------------------------------------
 total_pedidos             | 2000                
 receita_total             | 5004168.13          
 ticket_medio_geral        | 2502.08             
 total_clientes            | 2000                
 pedidos_por_cliente_medio | 1.0                 
 total_produtos            | 21                  
 total_unidades_vendidas   | 5007                
 cidades_atendidas         | 109                 
 estados_atendidos         | 22                  
 data_primeiro_pedido      | 2025-02-16 11:43:13 
 data_ultimo_pedido        | 2025-05-17 10:46:34 



In [0]:
print("\n🛠️ CORRIGINDO A TRANSFORMAÇÃO DOS STATUS")
print("=" * 50)

# Recriar a tabela fat_pedido_corrigida com os status ORIGINAIS corretos
spark.sql("""
CREATE OR REPLACE TABLE projeto_estatistica.silver.fat_pedido_corrigida AS
SELECT 
    fp.id,
    fp.data_pedido,
    fp.desconto,
    fp.subtotal,
    fp.total,
    -- Manter forma_pagamento corrigida
    fp.forma_pagamento,
    -- MANTER OS STATUS ORIGINAIS da bronze (CORRETOS)
    fo.Purchase_Status as status_compra,
    fp.id_cliente,
    fp.id_entrega
FROM (
    SELECT 
        id,
        data_pedido,
        desconto,
        subtotal,
        total,
        forma_pagamento,
        id_cliente,
        id_entrega
    FROM projeto_estatistica.silver.fat_pedido_corrigida
) fp
JOIN projeto_estatistica.bronze.fact_orders fo ON fp.id = fo.Id
""")

print("✅ Status corrigidos para os valores originais!")

# Verificar a correção
print("\n📋 STATUS APÓS CORREÇÃO:")
spark.sql("""
SELECT 
    status_compra,
    COUNT(*) as total_pedidos,
    ROUND((COUNT(*) / (SELECT COUNT(*) FROM projeto_estatistica.silver.fat_pedido_corrigida) * 100), 2) as percentual
FROM projeto_estatistica.silver.fat_pedido_corrigida
GROUP BY status_compra
ORDER BY total_pedidos DESC
""").show(truncate=False)


🛠️ CORRIGINDO A TRANSFORMAÇÃO DOS STATUS
✅ Status corrigidos para os valores originais!

📋 STATUS APÓS CORREÇÃO:
+-------------+-------------+----------+
|status_compra|total_pedidos|percentual|
+-------------+-------------+----------+
|Processando  |524          |26.2      |
|Cancelado    |516          |25.8      |
|Confirmado   |498          |24.9      |
|Em Analise   |462          |23.1      |
+-------------+-------------+----------+



In [0]:
print("\n💳 CONVERSÃO POR PAGAMENTO")
print("=" * 60)

# Recriar a análise de conversão com status CORRETOS
spark.sql("""
CREATE OR REPLACE TABLE projeto_estatistica.gold.conversao_pagamento_corrigida AS
SELECT 
    forma_pagamento,
    COUNT(*) as total_pedidos,
    
    -- Pedidos CONFIRMADOS (baseado nos status ORIGINAIS)
    SUM(CASE 
        WHEN LOWER(status_compra) = 'confirmado' THEN 1 
        ELSE 0 
    END) as pedidos_confirmados,
    
    -- Pedidos CANCELADOS
    SUM(CASE 
        WHEN LOWER(status_compra) = 'cancelado' THEN 1 
        ELSE 0 
    END) as pedidos_cancelados,
    
    -- Pedidos EM PROCESSAMENTO/ANÁLISE
    SUM(CASE 
        WHEN LOWER(status_compra) IN ('processando', 'em analise') THEN 1 
        ELSE 0 
    END) as pedidos_em_processo,
    
    -- TAXA DE CONVERSÃO (Confirmado vs Total)
    ROUND((SUM(CASE 
        WHEN LOWER(status_compra) = 'confirmado' THEN 1 
        ELSE 0 
    END) / COUNT(*) * 100), 2) as taxa_confirmacao_percentual,
    
    -- TAXA DE CANCELAMENTO
    ROUND((SUM(CASE 
        WHEN LOWER(status_compra) = 'cancelado' THEN 1 
        ELSE 0 
    END) / COUNT(*) * 100), 2) as taxa_cancelamento_percentual,
    
    -- TAXA DE PEDIDOS EM PROCESSO
    ROUND((SUM(CASE 
        WHEN LOWER(status_compra) IN ('processando', 'em analise') THEN 1 
        ELSE 0 
    END) / COUNT(*) * 100), 2) as taxa_processamento_percentual,
    
    -- VALORES MÉDIOS
    ROUND(AVG(CASE 
        WHEN LOWER(status_compra) = 'confirmado' THEN total 
        ELSE NULL 
    END), 2) as ticket_medio_confirmado,
    
    ROUND(AVG(CASE 
        WHEN LOWER(status_compra) = 'cancelado' THEN total 
        ELSE NULL 
    END), 2) as ticket_medio_cancelado,
    
    ROUND(AVG(CASE 
        WHEN LOWER(status_compra) IN ('processando', 'em analise') THEN total 
        ELSE NULL 
    END), 2) as ticket_medio_processo,
    
    -- RECEITAS
    ROUND(SUM(CASE 
        WHEN LOWER(status_compra) = 'confirmado' THEN total 
        ELSE 0 
    END), 2) as receita_confirmada,
    
    ROUND(SUM(CASE 
        WHEN LOWER(status_compra) = 'cancelado' THEN total 
        ELSE 0 
    END), 2) as receita_perdida,
    
    ROUND(SUM(CASE 
        WHEN LOWER(status_compra) IN ('processando', 'em analise') THEN total 
        ELSE 0 
    END), 2) as receita_potencial

FROM projeto_estatistica.silver.fat_pedido_corrigida
GROUP BY forma_pagamento
ORDER BY taxa_confirmacao_percentual DESC
""")

print("✅ conversao_pagamento_corrigida atualizada!")

# Mostrar resultado CORRETO
print("\n📊 CONVERSÃO POR PAGAMENTO - RESULTADO CORRETO:")
spark.sql("""
SELECT 
    forma_pagamento,
    total_pedidos,
    pedidos_confirmados,
    pedidos_cancelados,
    pedidos_em_processo,
    taxa_confirmacao_percentual as conversao_percentual,
    taxa_cancelamento_percentual as cancelamento_percentual,
    taxa_processamento_percentual as processamento_percentual,
    ticket_medio_confirmado,
    receita_confirmada
FROM projeto_estatistica.gold.conversao_pagamento_corrigida
ORDER BY conversao_percentual DESC
""").show(truncate=False)

# Atualizar KPIs principais
print("\n🔄 ATUALIZANDO KPIs PRINCIPAIS:")
spark.sql("""
CREATE OR REPLACE TABLE projeto_estatistica.gold.kpis_completos AS
SELECT 
    -- Manter todas as métricas anteriores
    receita_total,
    subtotal_total,
    desconto_total,
    desconto_medio,
    ticket_medio,
    total_frete,
    frete_medio,
    take_rate_frete_percentual,
    prazo_entrega_medio_dias,
    atraso_medio_dias,
    percentual_entregas_atrasadas,
    -- Atualizar com conversão CORRETA
    (SELECT ROUND(AVG(taxa_confirmacao_percentual), 2) 
     FROM projeto_estatistica.gold.conversao_pagamento_corrigida) as taxa_confirmacao_media,
    (SELECT SUM(pedidos_confirmados) 
     FROM projeto_estatistica.gold.conversao_pagamento_corrigida) as total_pedidos_confirmados,
    (SELECT SUM(pedidos_cancelados) 
     FROM projeto_estatistica.gold.conversao_pagamento_corrigida) as total_pedidos_cancelados,
    -- Manter o restante
    entregas_standard,
    entregas_same_day,
    entregas_scheduled,
    elasticidade_desconto_vendas,
    variacao_sazonal_percentual,
    total_pedidos,
    total_clientes,
    total_produtos,
    total_regioes,
    data_calculo
FROM projeto_estatistica.gold.kpis_completos
""")

print("✅ KPIs atualizados com conversão correta!")

# Mostrar resumo final
print("\n🎯 RESUMO FINAL - CONVERSÃO CORRETA:")
spark.sql("""
SELECT 
    total_pedidos,
    total_pedidos_confirmados,
    total_pedidos_cancelados,
    ROUND(taxa_confirmacao_media, 2) as taxa_confirmacao_media_percentual,
    ROUND((total_pedidos_cancelados / total_pedidos * 100), 2) as taxa_cancelamento_media_percentual,
    ROUND((SELECT SUM(pedidos_em_processo) FROM projeto_estatistica.gold.conversao_pagamento_corrigida) / total_pedidos * 100, 2) as taxa_processamento_media_percentual
FROM projeto_estatistica.gold.kpis_completos
""").show(vertical=True, truncate=False)


💳 CONVERSÃO POR PAGAMENTO
✅ conversao_pagamento_corrigida atualizada!

📊 CONVERSÃO POR PAGAMENTO - RESULTADO CORRETO:
+---------------+-------------+-------------------+------------------+-------------------+--------------------+-----------------------+------------------------+-----------------------+------------------+
|forma_pagamento|total_pedidos|pedidos_confirmados|pedidos_cancelados|pedidos_em_processo|conversao_percentual|cancelamento_percentual|processamento_percentual|ticket_medio_confirmado|receita_confirmada|
+---------------+-------------+-------------------+------------------+-------------------+--------------------+-----------------------+------------------------+-----------------------+------------------+
|pix            |487          |133                |123               |231                |27.31               |25.26                  |47.43                   |2783.67                |370228.58         |
|outros         |1003         |247                |256           

In [0]:
print("\n🎯 KPIs COMPLETOS")
print("=" * 65)

# KPIs COMPLETOS
spark.sql("""
CREATE OR REPLACE TABLE projeto_estatistica.gold.kpis_completos AS

WITH metricas_entrega AS (
    SELECT 
        de.id_entrega,
        de.servico,
        de.preco_servico,
        de.previsao_entrega,
        de.data_entrega,
        de.status,
        fp.data_pedido,
        fp.total,
        -- Prazo de entrega (dias)
        DATEDIFF(de.data_entrega, fp.data_pedido) as prazo_entrega_dias,
        -- Atraso (sim/não)
        CASE WHEN de.data_entrega > de.previsao_entrega THEN 1 ELSE 0 END as atraso,
        -- Dias de atraso
        CASE 
            WHEN de.data_entrega > de.previsao_entrega 
            THEN DATEDIFF(de.data_entrega, de.previsao_entrega)
            ELSE 0 
        END as dias_atraso
    FROM projeto_estatistica.silver.dim_entrega de
    JOIN projeto_estatistica.silver.fat_pedido_corrigida fp ON de.id_entrega = fp.id_entrega
    WHERE de.data_entrega IS NOT NULL AND de.previsao_entrega IS NOT NULL
),

metricas_pagamento AS (
    SELECT 
        forma_pagamento,
        COUNT(*) as total_pedidos,
        -- USANDO STATUS CORRETOS: Confirmado, Cancelado, Processando, Em Analise
        SUM(CASE WHEN LOWER(status_compra) = 'confirmado' THEN 1 ELSE 0 END) as pedidos_confirmados,
        SUM(CASE WHEN LOWER(status_compra) = 'cancelado' THEN 1 ELSE 0 END) as pedidos_cancelados,
        SUM(CASE WHEN LOWER(status_compra) IN ('processando', 'em analise') THEN 1 ELSE 0 END) as pedidos_em_processo,
        ROUND((SUM(CASE WHEN LOWER(status_compra) = 'confirmado' THEN 1 ELSE 0 END) / COUNT(*) * 100), 2) as taxa_confirmacao_percentual
    FROM projeto_estatistica.silver.fat_pedido_corrigida
    GROUP BY forma_pagamento
),

metricas_produtos AS (
    SELECT 
        dp.categoria,
        dp.subcategoria,
        COUNT(DISTINCT dc.id) as total_vendas,
        SUM(dc.quantidade) as unidades_vendidas,
        ROUND(SUM(dc.preco * dc.quantidade), 2) as receita_total,
        ROUND(AVG(fp.desconto), 4) as desconto_medio,
        ROUND((AVG(fp.desconto) / AVG(fp.subtotal)) * 100, 2) as desconto_percentual_medio
    FROM projeto_estatistica.silver.dim_compra_final dc
    JOIN projeto_estatistica.silver.dim_produto dp ON dc.id_produto = dp.id_produto
    JOIN projeto_estatistica.silver.fat_pedido_corrigida fp ON dc.id = fp.id
    GROUP BY dp.categoria, dp.subcategoria
),

metricas_sazonais AS (
    SELECT 
        DATE_TRUNC('month', fp.data_pedido) as mes,
        dc.regiao,
        dc.estado,
        COUNT(*) as total_pedidos,
        ROUND(SUM(fp.total), 2) as receita_total,
        ROUND(AVG(fp.total), 2) as ticket_medio
    FROM projeto_estatistica.silver.fat_pedido_corrigida fp
    JOIN projeto_estatistica.silver.dim_cliente dc ON fp.id_cliente = dc.id_cliente
    GROUP BY DATE_TRUNC('month', fp.data_pedido), dc.regiao, dc.estado
)

-- PRINCIPAIS KPIs
SELECT 
    -- 1. MÉTRICAS FINANCEIRAS BÁSICAS
    (SELECT ROUND(SUM(total), 2) FROM projeto_estatistica.silver.fat_pedido_corrigida) as receita_total,
    (SELECT ROUND(SUM(subtotal), 2) FROM projeto_estatistica.silver.fat_pedido_corrigida) as subtotal_total,
    (SELECT ROUND(SUM(desconto), 2) FROM projeto_estatistica.silver.fat_pedido_corrigida) as desconto_total,
    (SELECT ROUND(AVG(desconto), 4) FROM projeto_estatistica.silver.fat_pedido_corrigida) as desconto_medio,
    (SELECT ROUND(AVG(total), 2) FROM projeto_estatistica.silver.fat_pedido_corrigida) as ticket_medio,
    
    -- 2. MÉTRICAS DE FRETE/SERVIÇO
    (SELECT ROUND(SUM(preco_servico), 2) FROM projeto_estatistica.silver.dim_entrega) as total_frete,
    (SELECT ROUND(AVG(preco_servico), 2) FROM projeto_estatistica.silver.dim_entrega) as frete_medio,
    ROUND((SELECT SUM(preco_servico) FROM projeto_estatistica.silver.dim_entrega) / 
          (SELECT SUM(total) FROM projeto_estatistica.silver.fat_pedido_corrigida) * 100, 2) as take_rate_frete_percentual,
    
    -- 3. MÉTRICAS DE ENTREGA
    (SELECT ROUND(AVG(prazo_entrega_dias), 2) FROM metricas_entrega) as prazo_entrega_medio_dias,
    (SELECT ROUND(AVG(dias_atraso), 2) FROM metricas_entrega WHERE dias_atraso > 0) as atraso_medio_dias,
    (SELECT ROUND((SUM(atraso) / COUNT(*) * 100), 2) FROM metricas_entrega) as percentual_entregas_atrasadas,
    
    -- 4. CONVERSÃO DE PAGAMENTO (MÉDIA GERAL) - USANDO DADOS CORRETOS
    (SELECT ROUND(AVG(taxa_confirmacao_percentual), 2) FROM metricas_pagamento) as conversao_pagamento_media,
    (SELECT SUM(pedidos_confirmados) FROM metricas_pagamento) as total_pedidos_confirmados,
    (SELECT SUM(pedidos_cancelados) FROM metricas_pagamento) as total_pedidos_cancelados,
    (SELECT SUM(pedidos_em_processo) FROM metricas_pagamento) as total_pedidos_em_processo,
    
    -- 5. PERFORMANCE POR SERVIÇO DE ENTREGA
    (SELECT COUNT(*) FROM metricas_entrega WHERE servico = 'STANDARD') as entregas_standard,
    (SELECT COUNT(*) FROM metricas_entrega WHERE servico = 'SAME-DAY') as entregas_same_day,
    (SELECT COUNT(*) FROM metricas_entrega WHERE servico = 'SCHEDULED') as entregas_scheduled,
    
    -- 6. ELASTICIDADE PRODUTO vs DESCONTO (CORRELAÇÃO)
    (SELECT 
        ROUND(CORR(desconto_percentual_medio, unidades_vendidas), 4) as correlacao_desconto_vendas
     FROM metricas_produtos
    ) as elasticidade_desconto_vendas,
    
    -- 7. SAZONALIDADE (VARIAÇÃO MENSAL)
    (SELECT 
        ROUND((MAX(receita_total) - MIN(receita_total)) / AVG(receita_total) * 100, 2)
     FROM metricas_sazonais
    ) as variacao_sazonal_percentual,

    -- 8. RESUMO ADICIONAL
    (SELECT COUNT(*) FROM projeto_estatistica.silver.fat_pedido_corrigida) as total_pedidos,
    (SELECT COUNT(DISTINCT id_cliente) FROM projeto_estatistica.silver.fat_pedido_corrigida) as total_clientes,
    (SELECT COUNT(DISTINCT id_produto) FROM projeto_estatistica.silver.dim_compra_final) as total_produtos,
    (SELECT COUNT(DISTINCT regiao) FROM projeto_estatistica.silver.dim_cliente) as total_regioes,
    CURRENT_TIMESTAMP() as data_calculo
""")

print("✅ kpis_completos criada com dados corrigidos!")

# Mostrar KPIs principais
print("\n📊 KPIs PRINCIPAIS - RESUMO EXECUTIVO (CORRIGIDO):")
spark.sql("SELECT * FROM projeto_estatistica.gold.kpis_completos").show(vertical=True, truncate=False)

# KPIs detalhados por categoria - ATUALIZADOS
print("\n📈 KPIs DETALHADOS POR CATEGORIA:")

print("\n1. 🚚 PERFORMANCE POR SERVIÇO DE ENTREGA:")
spark.sql("""
SELECT 
    servico,
    COUNT(*) as total_entregas,
    ROUND(AVG(preco_servico), 2) as frete_medio,
    ROUND(AVG(prazo_entrega_dias), 2) as prazo_medio_dias,
    ROUND((SUM(CASE WHEN atraso = 1 THEN 1 ELSE 0 END) / COUNT(*) * 100), 2) as taxa_atraso_percentual
FROM (
    SELECT 
        de.servico,
        de.preco_servico,
        DATEDIFF(de.data_entrega, fp.data_pedido) as prazo_entrega_dias,
        CASE WHEN de.data_entrega > de.previsao_entrega THEN 1 ELSE 0 END as atraso
    FROM projeto_estatistica.silver.dim_entrega de
    JOIN projeto_estatistica.silver.fat_pedido_corrigida fp ON de.id_entrega = fp.id_entrega
    WHERE de.data_entrega IS NOT NULL
)
GROUP BY servico
ORDER BY total_entregas DESC
""").show(truncate=False)

print("\n2. 💳 CONVERSÃO POR FORMA DE PAGAMENTO (CORRIGIDA):")
spark.sql("""
SELECT 
    forma_pagamento,
    total_pedidos,
    pedidos_confirmados,
    pedidos_cancelados,
    pedidos_em_processo,
    taxa_confirmacao_percentual as conversao_percentual,
    taxa_cancelamento_percentual as cancelamento_percentual,
    taxa_processamento_percentual as processamento_percentual
FROM projeto_estatistica.gold.conversao_pagamento_corrigida
ORDER BY conversao_percentual DESC
""").show(truncate=False)

print("\n3. 📦 MIX POR CATEGORIA/SUBCATEGORIA:")
spark.sql("""
SELECT 
    categoria,
    subcategoria,
    total_vendas,
    unidades_vendidas,
    receita_total,
    desconto_medio,
    desconto_percentual_medio
FROM (
    SELECT 
        dp.categoria,
        dp.subcategoria,
        COUNT(DISTINCT dc.id) as total_vendas,
        SUM(dc.quantidade) as unidades_vendidas,
        ROUND(SUM(dc.preco * dc.quantidade), 2) as receita_total,
        ROUND(AVG(fp.desconto), 4) as desconto_medio,
        ROUND((AVG(fp.desconto) / AVG(fp.subtotal)) * 100, 2) as desconto_percentual_medio
    FROM projeto_estatistica.silver.dim_compra_final dc
    JOIN projeto_estatistica.silver.dim_produto dp ON dc.id_produto = dp.id_produto
    JOIN projeto_estatistica.silver.fat_pedido_corrigida fp ON dc.id = fp.id
    GROUP BY dp.categoria, dp.subcategoria
)
ORDER BY receita_total DESC
LIMIT 10
""").show(truncate=False)

print("\n4. 📊 ANÁLISE DE ELASTICIDADE vs DESCONTO:")
spark.sql("""
WITH elasticidade_categorias AS (
    SELECT 
        dp.categoria,
        dp.subcategoria,
        COUNT(DISTINCT dc.id) as total_vendas,
        SUM(dc.quantidade) as unidades_vendidas,
        ROUND(SUM(dc.preco * dc.quantidade), 2) as receita_total,
        ROUND(AVG(fp.desconto), 4) as desconto_medio,
        ROUND((AVG(fp.desconto) / AVG(fp.subtotal)) * 100, 2) as desconto_percentual_medio,
        -- Classificação de elasticidade
        CASE 
            WHEN (SUM(dc.quantidade) / COUNT(DISTINCT dc.id)) > 2.5 THEN 'ALTA ELASTICIDADE'
            WHEN (SUM(dc.quantidade) / COUNT(DISTINCT dc.id)) > 1.5 THEN 'MÉDIA ELASTICIDADE' 
            ELSE 'BAIXA ELASTICIDADE'
        END as nivel_elasticidade
    FROM projeto_estatistica.silver.dim_compra_final dc
    JOIN projeto_estatistica.silver.dim_produto dp ON dc.id_produto = dp.id_produto
    JOIN projeto_estatistica.silver.fat_pedido_corrigida fp ON dc.id = fp.id
    GROUP BY dp.categoria, dp.subcategoria
)

SELECT 
    categoria,
    subcategoria,
    total_vendas,
    unidades_vendidas,
    receita_total,
    desconto_medio,
    desconto_percentual_medio,
    nivel_elasticidade,
    ROUND((unidades_vendidas / total_vendas), 2) as unidades_por_venda
FROM elasticidade_categorias
ORDER BY unidades_por_venda DESC
LIMIT 15
""").show(truncate=False)

print("\n5. 📅 SAZONALIDADE POR MÊS/UF/REGIÃO:")
spark.sql("""
WITH dados_reais AS (
    SELECT 
        DATE_TRUNC('month', fp.data_pedido) as mes,
        dc.regiao,
        dc.estado,
        COUNT(*) as total_pedidos,
        ROUND(SUM(fp.total), 2) as receita_total,
        ROUND(AVG(fp.total), 2) as ticket_medio
    FROM projeto_estatistica.silver.fat_pedido_corrigida fp
    JOIN projeto_estatistica.silver.dim_cliente dc ON fp.id_cliente = dc.id_cliente
    GROUP BY DATE_TRUNC('month', fp.data_pedido), dc.regiao, dc.estado
)

SELECT 
    DATE_FORMAT(mes, 'yyyy-MM') as mes,
    regiao,
    estado,
    total_pedidos,
    receita_total,
    ticket_medio
FROM dados_reais
ORDER BY mes, regiao, estado
""").show(50,truncate=False) 

# Versão resumida por mês 
print("\n📊 RESUMO SAZONAL POR MÊS:")
spark.sql("""
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', data_pedido), 'yyyy-MM') as mes,
    DATE_FORMAT(DATE_TRUNC('month', data_pedido), 'MMMM') as mes_nome,
    COUNT(*) as total_pedidos,
    ROUND(SUM(total), 2) as receita_total,
    ROUND(AVG(total), 2) as ticket_medio,
    COUNT(DISTINCT id_cliente) as clientes_ativos
FROM projeto_estatistica.silver.fat_pedido_corrigida
GROUP BY DATE_TRUNC('month', data_pedido)
ORDER BY mes
""").show(truncate=False)

# Top 5 UFs por receita para cada mês 
print("\n🏆 TOP 5 UFs POR RECEITA A CADA MÊS:")
spark.sql("""
WITH ranked_ufs AS (
    SELECT 
        DATE_FORMAT(DATE_TRUNC('month', fp.data_pedido), 'yyyy-MM') as mes,
        dc.estado,
        dc.regiao,
        COUNT(*) as total_pedidos,
        ROUND(SUM(fp.total), 2) as receita_total,
        ROW_NUMBER() OVER (PARTITION BY DATE_TRUNC('month', fp.data_pedido) ORDER BY SUM(fp.total) DESC) as rank_uf
    FROM projeto_estatistica.silver.fat_pedido_corrigida fp
    JOIN projeto_estatistica.silver.dim_cliente dc ON fp.id_cliente = dc.id_cliente
    GROUP BY DATE_TRUNC('month', fp.data_pedido), dc.estado, dc.regiao
)

SELECT 
    mes,
    estado,
    regiao,
    total_pedidos,
    receita_total,
    rank_uf as posicao
FROM ranked_ufs
WHERE rank_uf <= 5
ORDER BY mes, rank_uf
""").show(truncate=False)

# Análise de crescimento mensal 
print("\n📈 CRESCIMENTO MENSAL - VARIAÇÃO PERCENTUAL:")
spark.sql("""
WITH vendas_mensais AS (
    SELECT 
        DATE_FORMAT(DATE_TRUNC('month', data_pedido), 'yyyy-MM') as mes,
        DATE_FORMAT(DATE_TRUNC('month', data_pedido), 'MMMM') as mes_nome,
        ROUND(SUM(total), 2) as receita_mensal,
        COUNT(*) as total_pedidos,
        LAG(ROUND(SUM(total), 2)) OVER (ORDER BY DATE_TRUNC('month', data_pedido)) as receita_mes_anterior,
        LAG(COUNT(*)) OVER (ORDER BY DATE_TRUNC('month', data_pedido)) as pedidos_mes_anterior
    FROM projeto_estatistica.silver.fat_pedido_corrigida
    GROUP BY DATE_TRUNC('month', data_pedido)
)

SELECT 
    mes,
    mes_nome,
    receita_mensal,
    total_pedidos,
    receita_mes_anterior,
    pedidos_mes_anterior,
    CASE 
        WHEN receita_mes_anterior IS NULL THEN '🔵 Primeiro mês'
        WHEN receita_mes_anterior = 0 THEN '🟡 Base zero'
        ELSE CONCAT('🟢 ', ROUND(((receita_mensal - receita_mes_anterior) / receita_mes_anterior * 100), 2), '%')
    END as variacao_receita,
    CASE 
        WHEN pedidos_mes_anterior IS NULL THEN '🔵 Primeiro mês'
        WHEN pedidos_mes_anterior = 0 THEN '🟡 Base zero'
        ELSE CONCAT('🟢 ', ROUND(((total_pedidos - pedidos_mes_anterior) / pedidos_mes_anterior * 100), 2), '%')
    END as variacao_pedidos
FROM vendas_mensais
ORDER BY mes
""").show(truncate=False)

# Distribuição percentual por mês
print("\n📋 DISTRIBUIÇÃO PERCENTUAL POR MÊS:")
spark.sql("""
WITH totais_gerais AS (
    SELECT 
        SUM(total) as receita_total_geral,
        COUNT(*) as total_pedidos_geral
    FROM projeto_estatistica.silver.fat_pedido_corrigida
),

vendas_por_mes AS (
    SELECT 
        DATE_FORMAT(DATE_TRUNC('month', data_pedido), 'yyyy-MM') as mes,
        DATE_FORMAT(DATE_TRUNC('month', data_pedido), 'MMMM') as mes_nome,
        ROUND(SUM(total), 2) as receita_mensal,
        COUNT(*) as total_pedidos
    FROM projeto_estatistica.silver.fat_pedido_corrigida
    GROUP BY DATE_TRUNC('month', data_pedido)
)

SELECT 
    vpm.mes,
    vpm.mes_nome,
    vpm.receita_mensal,
    vpm.total_pedidos,
    ROUND((vpm.receita_mensal / tg.receita_total_geral * 100), 2) as participacao_receita,
    ROUND((vpm.total_pedidos / tg.total_pedidos_geral * 100), 2) as participacao_pedidos
FROM vendas_por_mes vpm
CROSS JOIN totais_gerais tg
ORDER BY vpm.mes
""").show(truncate=False)


🎯 KPIs COMPLETOS
✅ kpis_completos criada com dados corrigidos!

📊 KPIs PRINCIPAIS - RESUMO EXECUTIVO (CORRIGIDO):
-RECORD 0---------------------------------------------------
 receita_total                 | 5004168.13                 
 subtotal_total                | 5343109.65                 
 desconto_total                | 148.42                     
 desconto_medio                | 0.0742                     
 ticket_medio                  | 2502.08                    
 total_frete                   | 65261.74                   
 frete_medio                   | 32.63                      
 take_rate_frete_percentual    | 1.3                        
 prazo_entrega_medio_dias      | 31.04                      
 atraso_medio_dias             | 3.04                       
 percentual_entregas_atrasadas | 83.25                      
 conversao_pagamento_media     | 25.03                      
 total_pedidos_confirmados     | 498                        
 total_pedidos_cancelados      